Example of mean hypothesis testing using the *Expected Encompassing Intrinsic Bayes Factor* (EEIBF) method described in [1].

The data set comes from a clinical trial by paramacologists Cushny and Peebles ([2]) at the Michigan Asylum for Insane at Kalamazoo (now Kalamazoo Psychiatric Hospital).

In the study ([3]), Cushny and Peebles dosed patients with small amount of different but related drugs and measured sleep activity to determine if any of the drugs could be an effective at inducing sleep. The data set was also studied by Student ([4]) and Fisher ([5]).

In this notebook, we'll use the same model that Student and Fisher used, but apply EEIBF hypothesis testing.


### References
1: Berger, J. and J. Mortera (1999). Default bayes factors for nonnested hypothesis testing. Journal of the American Statistical Association 94 (446), 542–554.
postscript for paper: https://www2.stat.duke.edu/~berger/papers/mortera.ps

2: Cushny, A. R. and Peebles, A. R. ‘The action of optical isomers. 11: Hyoscines’, Journal of Physiology, 32,
501-510 (1905).

3: Senn S, Richardson W. The first t-test. Stat Med. 1994 Apr 30;13(8):785-803. doi: 10.1002/sim.4780130802. PMID: 8047737.
pdf: http://www.medicine.mcgill.ca/epidemiology/hanley/bios601/Mean-Quantile/First.t.test.pdf

4: Student. ‘The probable error of a mean’, Biometrika VI (1908); reprinted in Pearson E. S. and Wishart J.
‘Student’s Collected Papers, The University Press, Cambridge, 1958 (citation uses reprint page numbers).

5: Fisher R. A. Statistical Methods for Research Workers, Oliver and Boyd, Edinburgh, 1925.

## Imports

In [1]:
import numpy as np
import pandas as pd
import scipy
from IPython.display import display, Math
from bbai.stat import NormalMeanHypothesis

## Data

In [2]:
# See page 791 of [3] for a copy of the original data.
sleep_a = np.array([1.3, 1.4, 4.5, 4.3, 6.1, 6.6, 6.2, 3.6, 1.1, 4.9, 6.3 ])
sleep_b = np.array([2.5, 3.8, 5.8, 5.6, 6.1, 7.6, 8.0, 4.4, 5.7, 6.3, 6.8 ])
delta = sleep_a - sleep_b
n = len(delta)
data = pd.DataFrame({
    'Patient': range(1, n+1),
    'Avg Sleep w/ L-hyoscyamine HBr' : sleep_a,
    'Avg Sleep w/ L-hyoscine HBr' : sleep_b,
    'Delta': delta
}).set_index('Patient')
data

,Avg Sleep w/ L-hyoscyamine HBr,Avg Sleep w/ L-hyoscine HBr,Delta
Patient,,,
1,1.3,2.5,-1.2
2,1.4,3.8,-2.4
3,4.5,5.8,-1.3
4,4.3,5.6,-1.3
5,6.1,6.1,0.0
6,6.6,7.6,-1.0
7,6.2,8.0,-1.8
8,3.6,4.4,-0.8
9,1.1,5.7,-4.6


In [3]:
data.describe()

,Avg Sleep w/ L-hyoscyamine HBr,Avg Sleep w/ L-hyoscine HBr,Delta
count,11.000000,11.000000,11.000000
mean,4.209091,5.690909,-1.481818
std,2.111613,1.618304,1.211460
min,1.100000,2.500000,-4.600000
25%,2.500000,5.000000,-1.600000
50%,4.500000,5.800000,-1.300000
75%,6.150000,6.550000,-0.900000
max,6.600000,8.000000,0.000000


## Test Hypothesis

Test the three hypotheses regarding the mean of delta
$$
\begin{align*}
H_0&: \mu = 0, \quad\quad\textrm{L-hyoscyamine HBr and L-hyscine HBr have the same effect}\\
H_-&: \mu < 0, \quad\quad\textrm{L-hyoscyamine HBr is less effective at inducing sleep than L-hyscine HBr}\\
H_+&: \mu > 0, \quad\quad\textrm{L-hyoscyamine HBr is more effective at inducing sleep than L-hyscine HBr}
\end{align*}
$$
We'll show how the posterior probabilities for the hypotheses evolve as more data is observed.

In [4]:
n = len(sleep_a)
probs_H_null = []
probs_H_minus = []
probs_H_plus = []
for i in range(n):
    if i < 2:
        probs_H_null.append(None)
        probs_H_minus.append(None)
        probs_H_plus.append(None)
        continue
    probs = NormalMeanHypothesis().test(delta[:i+1])
    probs_H_null.append(probs.equal)
    probs_H_minus.append(probs.left)
    probs_H_plus.append(probs.right)
pd.DataFrame({
    'n': range(1, n+1),
    'delta': delta,
    'H_0': probs_H_null,
    'H_-': probs_H_minus,
    'H_+': probs_H_plus
}).set_index('n')

,delta,H_0,H_-,H_+
n,,,,
1,-1.2,NaN,NaN,NaN
2,-2.4,NaN,NaN,NaN
3,-1.3,0.336883,0.472277,0.190840
4,-1.3,0.193446,0.734020,0.072534
5,0.0,0.213953,0.704815,0.081232
6,-1.0,0.132711,0.827604,0.039685
7,-1.8,0.061951,0.923300,0.014749
8,-0.8,0.034256,0.958680,0.007064
9,-4.6,0.073049,0.911635,0.015316
